In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('A Track 학습셋.csv')
data


,Log_Number,payload,label_action
0,0,"GET /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,1,POST /owa/auth/logon.aspx?replaceCurrent=1%22%...,System_Cmd_Execution
2,2,GET /goods/goods_search?display_type=list&arr_...,SQL_Injection
3,3,GET / HTTP/1.1\r\n\r\n,HOST_Scan
4,4,GET /sub_04_1_read.php?page=1&id=31%29%3BSELEC...,System_Cmd_Execution
...,...,...,...
44995,44995,GET /app/javascript/js/skin-responsive.js?v=%3...,System_Cmd_Execution
44996,44996,GET /board/?category=type+%25SYSTEMROOT%25%5Cw...,System_Cmd_Execution
44997,44997,POST /sub_02_4.php HTTP/1.1\r\nContent-Length:...,SQL_Injection
44998,44998,GET /admin/myadmin HTTP/1.1\r\nHost: lab.colle...,Path_Disclosure


In [3]:
data = data[['payload', 'label_action']]
data

,payload,label_action
0,"GET /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,POST /owa/auth/logon.aspx?replaceCurrent=1%22%...,System_Cmd_Execution
2,GET /goods/goods_search?display_type=list&arr_...,SQL_Injection
3,GET / HTTP/1.1\r\n\r\n,HOST_Scan
4,GET /sub_04_1_read.php?page=1&id=31%29%3BSELEC...,System_Cmd_Execution
...,...,...
44995,GET /app/javascript/js/skin-responsive.js?v=%3...,System_Cmd_Execution
44996,GET /board/?category=type+%25SYSTEMROOT%25%5Cw...,System_Cmd_Execution
44997,POST /sub_02_4.php HTTP/1.1\r\nContent-Length:...,SQL_Injection
44998,GET /admin/myadmin HTTP/1.1\r\nHost: lab.colle...,Path_Disclosure


In [4]:
data = data[(data['label_action'] == 'Cross_Site_Scripting')]
data

,payload,label_action
15,GET /promotion/event?event_name=ZAP&sc_filter=...,Cross_Site_Scripting
63,GET /board/?category=&gdviewer=1&goods_seq=69&...,Cross_Site_Scripting
119,GET /index.php/content/search/?SectionID=3&Sea...,Cross_Site_Scripting
131,GET /board/board_list?code=%3Cscript%3Eprompt%...,Cross_Site_Scripting
135,GET /board/board_write?code=%3Cscript%3Eprompt...,Cross_Site_Scripting
...,...,...
44717,GET /goods/brand?code=0002&event=&filter_displ...,Cross_Site_Scripting
44718,GET /member/login?return_url=javascript:alert(...,Cross_Site_Scripting
44743,GET /board/board_view?code=%3Cscript%3Eprompt(...,Cross_Site_Scripting
44796,GET /promotion/event?sc_status=ing&sc_filter=s...,Cross_Site_Scripting


In [5]:
data.to_json('Cross_Site_Scripting.json', 'records')


In [6]:
import json

In [7]:
with open('Cross_Site_Scripting.json', 'r') as f:
    data = json.load(f)


In [8]:
import urllib.parse  #URL 처리와 관련된 여러 유용한 함수와 클래스 제공

def unquote(text):
    k = 0               #반복 회수를 세기 위한 변수 k
    uq_prev = text      #uq_prev text 인자 받은거 입력
    while(k < 100):     #반복 작업을 100회 이내로 실행
        uq = urllib.parse.unquote_plus(uq_prev)
        if uq == uq_prev:
            break
        else:
            uq_prev = uq
    
    return uq_prev

def remove_new_line(text):
    text = text.strip()
    text = ' '.join(text.splitlines())
    
    return text

def remove_multiple_whitespaces(text):
    return ' '.join(text.split())

def clean_pattern(pattern):
    pattern = unquote(pattern)                      #url decoding
    pattern = remove_new_line(pattern)              #\n 줄바꿈 문자 제거
    pattern = pattern.lower()                       #소문자로 바꾸는거
    pattern = remove_multiple_whitespaces(pattern)  # 공백 제거
    
    return pattern


def prepare_data(x):
    out = {}            #var dict
    out['pattern'] = clean_pattern(x['payload'])
    
    if x['label_action'] == 'Cross_Site_Scripting':
        out['label_action'] = 'Cross_Site_Scripting'
    
    return out

In [9]:
Cross_Site_Scripting = list(map(prepare_data, data))
Cross_Site_Scripting

[{'pattern': 'get /promotion/event?event_name=zap&sc_filter=sales&sc_status=</script><img src=x onerror=prompt()><script> http/1.1\\r\\nhost: www.mall.shop\\r\\nuser-agent: mozilla/5.0 (windows nt 10.0; win64; x64; rv:92.0) gecko/20100101 firefox/92.0\\r\\npragma: no-cache\\r\\ncache-control: no-cache\\r\\nreferer: http://www.mall.shop/promotion/event\\r\\ncookie: shopreferer=; today_view=a:15:{i:0;i:38;i:1;i:47;i:2;i:41;i:3;i:56;i:4;i:51;i:5;i:42;i:6;i:79;i:7;i:43;i:8;i:77;i:9;i:75;i:10;i:68;i:11;i:54;i:12;i:70;i:13;i:55;i:14;i:52;}; marketplace=direct\\r\\ncontent-length: 0\\r\\n\\r\\n',
  'label_action': 'Cross_Site_Scripting'},
 {'pattern': 'get /board/?category=&gdviewer=1&goods_seq=69&id=goods_qna&iframe=1&page=<script>alert(1)</script>&perpage=&popup=&search_text=zap http/1.1\\r\\nhost: www.mall.shop\\r\\nuser-agent: mozilla/5.0 (x11; linux x86_64; rv:91.0) gecko/20100101 firefox/91.0\\r\\naccept: text/html,application/xhtml xml,application/xml;q=0.9,image/webp,*/*;q=0.8\\r\\nac

In [10]:
with open('Cross_Site_Scripting.json', 'w') as f:
    json.dump(Cross_Site_Scripting, f)